In [1]:
import tensorflow as tf

In [2]:
print(tf.math.add(1, 2))
print(tf.math.add([1, 2], [3, 4]))
print(tf.math.square(5))
print(tf.math.reduce_sum([1, 2, 3]))

# Operator overloading is also supported
print(tf.math.square(2) + tf.math.square(3))

tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor([4 6], shape=(2,), dtype=int32)
tf.Tensor(25, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(13, shape=(), dtype=int32)


In [4]:

x = tf.linalg.matmul([[1]], [[2, 3]]) # function performs matrix multiplication between two tensors.
print(x)
print(x.shape)
print(x.dtype)

tf.Tensor([[2 3]], shape=(1, 2), dtype=int32)
(1, 2)
<dtype: 'int32'>


In [5]:
import numpy as np

ndarray = np.ones([3, 3])

print("TensorFlow operations convert numpy arrays to Tensors automatically")
tensor = tf.math.multiply(ndarray, 42)
print(tensor)


print("And NumPy operations convert Tensors to NumPy arrays automatically")
print(np.add(tensor, 1))

print("The .numpy() method explicitly converts a Tensor to a numpy array")
print(tensor.numpy())

TensorFlow operations convert numpy arrays to Tensors automatically
tf.Tensor(
[[42. 42. 42.]
 [42. 42. 42.]
 [42. 42. 42.]], shape=(3, 3), dtype=float64)
And NumPy operations convert Tensors to NumPy arrays automatically
[[43. 43. 43.]
 [43. 43. 43.]
 [43. 43. 43.]]
The .numpy() method explicitly converts a Tensor to a numpy array
[[42. 42. 42.]
 [42. 42. 42.]
 [42. 42. 42.]]


In [7]:
# generates a tensor 3*3 of random values sampled from a uniform distribution within a specific range
x = tf.random.uniform([3, 3])

# list of PhysicalDevice objects if a GPU is available, indicating its name and memory capacity.
print("Is there a GPU available: "),
print(tf.config.list_physical_devices("GPU"))

print("Is the Tensor on GPU #0:  "),
print(x.device.endswith('GPU:0'))
x

Is there a GPU available: 
[]
Is the Tensor on GPU #0:  
False


<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[0.13341367, 0.00588524, 0.42831945],
       [0.39547944, 0.841665  , 0.10192502],
       [0.92636454, 0.82545495, 0.970448  ]], dtype=float32)>

In [8]:
import time

def time_matmul(x):
  start = time.time()
  for loop in range(10):
    tf.linalg.matmul(x, x)

  result = time.time()-start

  print("10 loops: {:0.2f}ms".format(1000*result))

# Force execution on CPU
print("On CPU:")
with tf.device("CPU:0"):
  x = tf.random.uniform([1000, 1000])
  assert x.device.endswith("CPU:0")
  time_matmul(x)

# Force execution on GPU #0 if available
if tf.config.list_physical_devices("GPU"):
  print("On GPU:")
  with tf.device("GPU:0"): # Or GPU:1 for the 2nd GPU, GPU:2 for the 3rd etc.
    # Asserts that x is indeed located on the CPU using assert x.device.endswith("CPU:0").
    x = tf.random.uniform([1000, 1000]) # 
    assert x.device.endswith("GPU:0")
    time_matmul(x)

On CPU:
10 loops: 108.41ms


In [12]:
ds_tensors = tf.data.Dataset.from_tensor_slices([1, 2, 3, 4, 5, 6])
ds_tensors

<_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>

In [9]:
# Create a CSV file
import tempfile
# Creates a temporary file and returns its filename as filename
_, filename = tempfile.mkstemp() # 

with open(filename, 'w') as f:
  f.write("""Line 1
Line 2
Line 3
  """)
# Creates a TensorFlow dataset named ds_file by reading the lines of
# text from the specified CSV file (filename).
# Each line in the file becomes an element (string tensor) in the dataset.
ds_file = tf.data.TextLineDataset(filename)

In [15]:
# .map(tf.math.square) applies the tf.math.square function to each element (integer) 
# in the ds_tensors dataset. This squares each value.
# .shuffle(2) shuffles the order of elements in the dataset randomly. 
# The argument 2 defines the buffer size used for shuffling.
# .batch(2) groups the elements into batches of 2. This means the 
# output will be a dataset of tensors, each containing two squared values
ds_tensors = ds_tensors.map(tf.math.square).shuffle(2).batch(2)
# the output will be a dataset of tensors, each containing two lines of text.
ds_file = ds_file.batch(2)

In [17]:
ds_tensors

<_BatchDataset element_spec=TensorSpec(shape=(None,), dtype=tf.int32, name=None)>

In [18]:
print('Elements of ds_tensors:')
for x in ds_tensors:
  print(x)

print('\nElements in ds_file:')
for x in ds_file:
  print(x)

Elements of ds_tensors:
tf.Tensor([4 9], shape=(2,), dtype=int32)
tf.Tensor([16 25], shape=(2,), dtype=int32)
tf.Tensor([36  1], shape=(2,), dtype=int32)

Elements in ds_file:
tf.Tensor(
[[b'Line 1' b'Line 2']
 [b'Line 3' b'  ']], shape=(2, 2), dtype=string)
